In [ ]:
# --- 🎨 Générateur d'art abstrait "Splash" pour le hackathon ---

import pandas as pd
from PIL import Image, ImageDraw
import random
import math
import numpy as np
from IPython.display import display

# ======================
# Étape 1 : Normalisation du CSV
# ======================
def normaliser_csv(df: pd.DataFrame) -> pd.DataFrame:
    """
    Essaie de remettre en ordre les colonnes d'un CSV désordonné :
    - Identifie les colonnes numériques et textuelles
    - Renomme celles utiles pour le dessin
    """
    # Séparation numérique / texte
    numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
    text_cols = df.select_dtypes(exclude=["number"]).columns.tolist()

    # Sécurité : s'assurer qu'on a assez de colonnes
    if len(numeric_cols) < 2:
        raise ValueError("Le CSV doit contenir au moins deux colonnes numériques.")

    # Recomposition du DataFrame standardisé
    new_df = pd.DataFrame()
    new_df["Categorie"] = df[text_cols[0]] if len(text_cols) > 0 else "Inconnue"
    new_df["Coordonnees"] = df[numeric_cols[0]]
    new_df["Couleur"] = df[numeric_cols[1]]

    return new_df


# ======================
# Étape 2 : Fond dégradé
# ======================
def create_gradient_background(width, height, top_color=(20, 20, 30), bottom_color=(80, 40, 100)):
    img = Image.new("RGB", (width, height), color=0)
    draw = ImageDraw.Draw(img)
    for y in range(height):
        ratio = y / height
        r = int(top_color[0] * (1 - ratio) + bottom_color[0] * ratio)
        g = int(top_color[1] * (1 - ratio) + bottom_color[1] * ratio)
        b = int(top_color[2] * (1 - ratio) + bottom_color[2] * ratio)
        draw.line([(0, y), (width, y)], fill=(r, g, b))
    return img


# ======================
# Étape 3 : Splash de peinture
# ======================
def draw_color_splash(draw: ImageDraw.Draw, x, y, base_color, intensity=1.0, radius=50):
    num_drops = int(100 * intensity)
    for _ in range(num_drops):
        angle = random.uniform(0, 2 * math.pi)
        dist = random.uniform(0, radius)
        px = x + math.cos(angle) * dist
        py = y + math.sin(angle) * dist

        r = min(255, max(0, base_color[0] + random.randint(-30, 30)))
        g = min(255, max(0, base_color[1] + random.randint(-30, 30)))
        b = min(255, max(0, base_color[2] + random.randint(-30, 30)))

        size = random.randint(2, 8)
        draw.ellipse((px - size, py - size, px + size, py + size), fill=(r, g, b))


# ======================
# Étape 4 : Génération de l'œuvre complète
# ======================
def generate_splash_art(csv_path="composition_2_explosion.csv", output_path="oeuvre_splash.png", width=1000, height=700):
    # Lecture CSV
    df = pd.read_csv(csv_path)
    df = normaliser_csv(df)

    # Fond dégradé
    background = create_gradient_background(width, height)
    image = background.convert("RGBA")
    draw = ImageDraw.Draw(image, "RGBA")

    # Normalisation des coordonnées
    min_coord = df["Coordonnees"].min()
    max_coord = df["Coordonnees"].max()
    coord_norm = (df["Coordonnees"] - min_coord) / (max_coord - min_coord + 1e-9)

    # Génération des splashs
    for i, row in df.iterrows():
        x = int(100 + coord_norm.iloc[i] * (width - 200))
        y = random.randint(100, height - 100)
        base = int(row["Couleur"])
        r = int(base % 256)
        g = int((base * 2) % 256)
        b = int((255 - base) % 256)

        # Splash principal
        radius = random.randint(50, 120)
        intensity = random.uniform(0.8, 1.4)
        draw_color_splash(draw, x, y, (r, g, b), intensity=intensity, radius=radius)

        # Coulures verticales
        for _ in range(random.randint(1, 3)):
            drip_x = x + random.randint(-20, 20)
            drip_y_end = min(height - 10, y + random.randint(50, 150))
            draw.line((drip_x, y, drip_x, drip_y_end), fill=(r, g, b, 180), width=random.randint(3, 6))

    # Effet lumière douce
    overlay = Image.new("RGBA", image.size, (255, 255, 255, 20))
    image = Image.alpha_composite(image, overlay)

    # Sauvegarde et affichage
    final = image.convert("RGB")
    final.save(output_path, quality=95)
    print(f"✅ Œuvre abstraite 'splash' générée : {output_path}")
    display(final)


# ======================
# Étape 5 : Exécution
# ======================
generate_splash_art("composition_2_explosion.csv", "oeuvre_splash.png")
